<a href="https://colab.research.google.com/github/jxxngho/HG_MLDL/blob/main/05_3_Tree_Ensembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 정형 데이터와 비정형 데이터

In [2]:
# 정형 데이터 : 데이터베이스에 저장할 수 있는 데이터( 행과 열이 구분된 데이터..), 레코드라고도 함. csv,엑셀에 저장
# 비정형 데이터 : 책의 글과 같은 텍스트 데이터, 디지털 카메라로 찍은 사진, 디지털 음악 등등 ...

# 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘은 앙상블 학습이다.
# 이 알고리즘은 대부분 결정 트리를 기반으로 만들어져 있음.

# 그럼 비정형 데이터에는 어떤 알고리즘을 사용해야하나?? -->> 신경망 알고리즘! (7장에서 배움)
# 비정형 데이터는 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 까다로움

# 랜덤 포레스트

In [3]:
# 랜덤 포레스트 : 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다
# 이 후 결정 트리의 예측을 사용해 최종 예측을 만든다.

# 랜덤 포레스트가 어떻게 숲을 구성???
# 부트스트랩 샘플로 뽑는다 !! -> 중복을 허용한 샘플링
# 부트스트랩 샘플로 결정 트리를 훈련하고 클래스별 확률을 다 더해서 트리 개수로 나눠준다.
# 각 클래스별로 평균적 확률이 나옴! -> 제일 큰 것 결정

# 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고
# 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('http://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2,random_state=42)

In [9]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1,random_state=42)

# return_train_score = True로 지정하면 검증 점수 뿐만 아니라 훈련 세트에 대한 점수도 함께 반환한다.
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [10]:
# 출력 결과가 훈련 세트에 다소 과대적합 ..ㅜ

In [11]:
# 특성 중요도 확인
rf.fit(train_input,train_target)
print(rf.feature_importances_)

# 앞에 1절 결정 트리에서 만든 특성 중요도와 다른 이유는?
# 랜덤 포레스트가 틀성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문 !
# 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음.
# !! 과대적합을 줄이고 일반화 성능을 높이는데 도움이 됨

[0.23167441 0.50039841 0.26792718]


In [12]:
# 자체적으로 모델을 평가하는 기능이 있다?
# 랜덤 포레스트는 부트스트랩 샘플에 포함되지 않고 남은 샘플(OOB샘플)로 결정 트리를 평가할 수 있다.

rf = RandomForestClassifier(oob_score=True, n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

# OOB점수를 사용하면 교차 검증을 대신할 수 있어 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다.

0.8934000384837406


# 엑스트라 트리

In [ ]:
# 엑스트라 트리는 랜덤 포레스트와 매우 비슷하게 동작
# 다른 점은 부트스트랩 샘플을 사용하지 않는다는 점!
# 전체 훈련 세트를 사용해서, 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할
# splitter = 'random'
# 하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블 하기 때문에
# 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다.

In [13]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et,train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [14]:
# 특성 중요도
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그레이디언트 부스팅

In [15]:
# 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식
# 경사 하강법을 사용하여 트리를 앙상블에 추가
# 학습률 매개변수로 속도를 조절할 수도 있음 ㅇㅅㅇa

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

# 그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강함!

0.8881086892152563 0.8720430147331015


In [19]:
# 결정트리 개수 5배 늘려보기
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

# 과대적합을 잘 억제하는 모습을 보임

0.9464595437171814 0.8780082549788999


In [21]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

# 그레이디언트 부스팅이 랜덤 포레스트보다 일부 특성(당도)에 더 집중하는것을 알 수 있음.

[0.15872278 0.68010884 0.16116839]


# 히스토그램 기반 그레이디언트 부스팅

In [33]:
# 입력 특성을 256개의 구간으로 나눈다.
# 256개 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb,train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [35]:
# 과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 좋은 성능을 제공해준다
# 특성 중요도 확인해보자! -> permutation_importance()
# 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여
# 어떤 특성이 중요한지를 계산
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

# 히스토그램 기반 그레이디언트 부스팅 모델을 훈련하고 훈련 세트에서 특성 중요도 계산
# n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정

# 반환 객체에는 특성 중요도, 평균, 표준 편차를 담고 있음.

# 두 번째 속성을 섞으면 정확도가 0.23만큼 떨어진다.. 그만큼 중요하다!

[0.08876275 0.23438522 0.08027708]


In [36]:
# 테스트 세트에서 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [37]:
# 테스트 세트에서의 성능 확인
hgb.score(test_input, test_target)

0.8723076923076923

# XGBoost


In [39]:
from re import X
# 사이킷런 말고도 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 많다
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist',random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


# LightGBM

In [40]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)

scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
